# Baseline LSTM Model and SBERT


In [11]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.autograd import Variable
from torch.nn import functional as F
from glob import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding Shape:", embedding.shape)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding Shape: (768,)

Sentence: Sentences are passed as a list of string.
Embedding Shape: (768,)

Sentence: The quick brown fox jumps over the lazy dog.
Embedding Shape: (768,)



## Data Loader for labeled episodes with embedded sentences

Need to have embeddings saved from Model_Pretraining

Initiating the class takes some time to load all the embeddings into memory. 



In [64]:
def ls_data_loader(path, train=True):
    '''loads labeled sub files, embeds the sentences, and saves
    embedded episodes and labels into binary files
    '''
    # get path to all labels_sub files
    sub_files = glob(path + '/*.tsv')
    
    # loads texts
    data = []
    labels = []

    if train:
        start=0
        stop=30
    else:
        start=30
        stop=-1
    for f in sub_files[start:stop]:
        print('loading:', f)
        # read with pandas
        df = pd.read_csv(f, sep='\t', usecols=['text', 'label'])
        # get sentences as list
        sentences = list(df["text"].values)
        # convert labels to 0 and 1
        tags = [0 if v == 'N' else 1 for v in df["label"]]
        data.append(sentences)
        labels.append(tags)
    return data, labels

In [67]:


class Dataset_seq_ep(torch.utils.data.Dataset):
    def __init__(self, train_path, train=True):
        # self.sent_id = sent_id
        self.train_path = train_path
        self.data, self.labels = ls_data_loader(train_path, train)
        self.embedder = SentenceTransformer('all-mpnet-base-v2')
        self.data = self.__embedder__(self.data)
        

    def __getitem__(self, index):
        # return sequence of sentences and labels
        seq = torch.Tensor(self.data[index])
        labels = torch.Tensor(self.labels[index])
        return seq, labels

    def __len__(self):
        return(len(self.data))

    def __embedder__(self, data):
        return [self.embedder.encode(sentences) for sentences in data]



def collate_fn(batch):
	'''  
	custom collate_fn as the size of every episode is different and merging sequences (including padding) 
	is not supported in default. 
	'''

	(xx, yy) = zip(*batch)
	x_lens = [len(x) for x in xx]
	y_lens = [len(y) for y in yy]

	xx_pad = pad_sequence(xx, batch_first=True, padding_value=-1)
	yy_pad = pad_sequence(yy, batch_first=True, padding_value=-1)

	return xx_pad, yy_pad, x_lens, y_lens

# train_dataset = Dataset_seq_ep('labeled_subs')
test_dataset = Dataset_seq_ep('labeled_subs', train=False)

test_dataloader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=collate_fn)
# train_dataloader = torch.utils.data.DataLoader(
#     dataset=train_dataset,
#     batch_size=1,
#     shuffle=True,
#     collate_fn=collate_fn)


    


loading: labeled_subs\s05e05.tsv
loading: labeled_subs\s05e06.tsv
loading: labeled_subs\s05e08.tsv
loading: labeled_subs\s05e10.tsv
loading: labeled_subs\s05e12.tsv
loading: labeled_subs\s05e13.tsv
loading: labeled_subs\s05e17.tsv
loading: labeled_subs\s05e21.tsv


Check dataloader and see that batches are padded to seq with max_length per batch and the length of each sequence is returned

In [56]:
for i in range(1):
    loop = tqdm(train_dataloader)
    for batch in loop:
        data, labels, in_len, lab_len = batch
        print('Data shape', data.shape)
        print('labels shape', labels.shape)
        break

100%|██████████| 30/30 [00:00<00:00, 123.44it/s]

Data shape torch.Size([1, 640, 768])
labels shape torch.Size([1, 640])
Data shape torch.Size([1, 647, 768])
labels shape torch.Size([1, 647])
Data shape torch.Size([1, 705, 768])
labels shape torch.Size([1, 705])
Data shape torch.Size([1, 688, 768])
labels shape torch.Size([1, 688])
Data shape torch.Size([1, 696, 768])
labels shape torch.Size([1, 696])
Data shape torch.Size([1, 587, 768])
labels shape torch.Size([1, 587])
Data shape torch.Size([1, 652, 768])
labels shape torch.Size([1, 652])
Data shape torch.Size([1, 612, 768])
labels shape torch.Size([1, 612])
Data shape torch.Size([1, 602, 768])
labels shape torch.Size([1, 602])
Data shape torch.Size([1, 701, 768])
labels shape torch.Size([1, 701])
Data shape torch.Size([1, 657, 768])
labels shape torch.Size([1, 657])
Data shape torch.Size([1, 597, 768])
labels shape torch.Size([1, 597])
Data shape torch.Size([1, 670, 768])
labels shape torch.Size([1, 670])
Data shape torch.Size([1, 743, 768])
labels shape torch.Size([1, 743])
Data s

### Build LSTM model

adapting from 

Blog post:
Taming LSTMs: Variable-sized mini-batches and why PyTorch is good for your health:
https://medium.com/@_willfalcon/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e



In [57]:

# this is still a work in progress

class PerpLSTM(nn.Module):
    def __init__(self, nb_lstm_layers=1, nb_lstm_units=128, fc_hidden_units=100, embedding_dim=3, batch_size=1):
        super(PerpLSTM, self).__init__()

        # self.vocab = {'<PAD>': -1} # not sure we need this
        # self.tags = {'<PAD>':-1, 'N': 0, 'Y': 1}

        self.nb_lstm_layers = nb_lstm_layers
        self.nb_lstm_units = nb_lstm_units
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.fc_hidden_units = fc_hidden_units

        # # don't count the padding tag for the classifier output
        # self.nb_tags = len(self.tags)-1

        # build actual NN
        self.__build_model()

    def __build_model(self):

        self.relu = nn.ReLU()

        # design LSTM
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.nb_lstm_units,
            num_layers=self.nb_lstm_layers,
            batch_first=True,
        )

        # output layer which projects back to tag space
        self.fc1 = nn.Linear(self.nb_lstm_units, self.fc_hidden_units)
        self.fc2 = nn.Linear(self.fc_hidden_units, 1) # change out to 1 for sigmoid activation


    def forward(self, X):

        # run through ReLu
        X = self.relu(X)

        # run through LSTM
        X, self.hidden = self.lstm(X)

        # reshape for linear
        # X = X.contiguous()
        # X = X.view(-1, X.shape[2])

        # run through linear layers and finish with sigmoid func
        X = self.fc1(X)
        X = self.relu(X)
        X = torch.sigmoid(self.fc2(X))

        return X


In [58]:

perp_model = PerpLSTM(
    nb_lstm_layers=1, 
    nb_lstm_units=128, 
    embedding_dim=768,
    batch_size=1)


## Testing forward

In [59]:
for i in range(1):
    loop = tqdm(train_dataloader)
    for batch in loop:
        data, labels, in_len, lab_len = batch
        print('Data shape', data.shape)
        print('labels shape', labels.shape)
        with torch.no_grad():
            output = perp_model.forward(data)
        print(output.shape)
        break

  0%|          | 0/30 [00:00<?, ?it/s]

Data shape torch.Size([1, 705, 768])
labels shape torch.Size([1, 705])
torch.Size([1, 705, 1])


## Testing backward

In [60]:

optimizer = torch.optim.Adam(perp_model.parameters(), lr=0.001) 
# Using Binary Cross Entropy Loss function since we are using batch size = 1
criterion = nn.BCELoss()

for epoch in range(100):
    loop = tqdm(train_dataloader)
    epoch_total = 0
    for batch in loop:
        data, labels, in_len, lab_len = batch

        outputs = perp_model.forward(data) #forward pass
        optimizer.zero_grad() #calculate the gradient, manually setting to 0
 
        # obtain the loss function
        # loss = perp_model.loss(outputs, labels, lab_len)
        # m = nn.Sigmoid()
        # sig_out = m(outputs)
        # print(sig_out.shape)

        # loss = criterion(outputs, F.one_hot(labels.view(-1).type(torch.int64)).type(torch.float32))
        loss = criterion(outputs.view(1,-1), labels.type(torch.float32))
        loss.backward() #calculates the loss of the loss function
        
        optimizer.step() #improve from loss, i.e backprop
        loop.set_postfix(loss=loss.item())
        epoch_total += loss.item()
    print("Epoch: %d, loss: %1.5f" % (epoch, epoch_total/len(loop)))


100%|██████████| 30/30 [00:11<00:00,  2.52it/s, loss=0.474]


Epoch: 0, loss: 0.52429


100%|██████████| 30/30 [00:11<00:00,  2.69it/s, loss=0.342]


Epoch: 1, loss: 0.42136


100%|██████████| 30/30 [00:11<00:00,  2.64it/s, loss=0.427]


...

Epoch: 97, loss: 0.00011


100%|██████████| 30/30 [00:12<00:00,  2.50it/s, loss=5.34e-5] 


Epoch: 98, loss: 0.00010


100%|██████████| 30/30 [00:11<00:00,  2.55it/s, loss=0.000129]

Epoch: 99, loss: 0.00010


In [61]:
print((perp_model(data)>.5).sum(1))
print(labels.sum(1))


tensor([[154]])
tensor([154.])
